## Generating an instance of the SPB-SFX instrument

FELpy extends the functionality of wpg.wavefront.Wavefront and wpg.beamline.Beamline classes, and adds extended, instrument specific functionality to the design and analysis of XFEL beamline data by the use of the Instrument class.

We have developed a high-level API to quickly retrieve the the SPB-SFX instrument class, which contains the wpg beamline structure.

Please note that this is still under developement, and some of the cracks may be obvious, in future we will pursuit a more rounded approach.

The following example addresses the case for a paticular paramater space of the instrument, which is loaded from a .json file. We note that the propagation parameters for each propagation step (in the wpg/srw sense) are sensitive to the properties of the wavefront and may need to be changed accordingly.

The setup_spb function is a wrapper for the Instrument class, which contains functions for loading and modifying instrument parameters, as well as building the beamline object.

## Initialising the SPB-SFX Instrument

In [1]:
from felpy.model.beamlines.exfel_spb.methods import setup_spb

In [2]:
spb_sfx = setup_spb(parameter_file = "/gpfs/exfel/data/user/guestt/FELpy/felpy/data/params/spb-sfx_nkb_FAST.json", ### This is the .json file containing all of the necessary parameters to describe the spb-sfx isntrument and propagation
                    options = 'nano', ### here we specify the nano-kb configuration, also accepts 'direct' and 'micron', although they are less developed
                    ekev = 5.0, ### we need to specify the approximate beam energy for configuring our mirrors
                    apertures = True, ### option to toggle mirror apertures on and off
                    surface = 'real', ### toggle for mirror surfaces - can be 'real' or 'flat'
                    crop = None, ### option to crop the beamline between two elements, takes list i.e. ["HOM1", "HOM2"] and returns truncated beamline
                    theta_HOM = 2.3e-03, ### nominal angle of Horizontal offset mirrors, can be changed independently afterwards
                    theta_KB = 3.5e-03, ### nominal angle of KB mirrors, can be changed independently afterwards
                    save_params = False) ### currently deprecated.


Initialising Single Particle Beamline


## Adjusting Mirrors

The above spb_sfx variable is an instance of the exfel_spb instrument class and contains some special functions for editing the initialised beamline model.
For example, we can orientation angle of a mirror as follows - note that we state the energy (in keV). Note: in order to commit these changes, we are required to rebuild the beamline:

In [3]:
spb_sfx.adjust_mirror(mirror_name = "HOM1",
                      ekev = 5.0,
                      new_ang = 3.5e-03)

spb_sfx.rebuild(focus = 'nano')

## Editing Beamline Elements
You can see all of the elements in the beamline in order:

In [4]:
spb_sfx.list_elements()

['d1',
 'HOM1',
 'd2',
 'HOM2',
 'd3',
 'NKB_PSlit',
 'd4',
 'NHE_error',
 'NHE',
 'd5',
 'NVE_error',
 'NVE']

There are two alternative methods of editing the beamline:

### 1. Editing Beamline Objects (Not reccomended).
The exfel_spb instrument class generates wpg.optical_elements type objects from the parameters file - we can edit them directly using the above listed elements, e.g:

In [9]:
spb_sfx.d1.L = 247 ### change the drift distance of the first drift element (d1) to 247 m
spb_sfx.build_beamline() ### rebuilds the wpg beamline from the updated object, note that calling spb_sfx.rebuild() will overwrite the prior change.

### 2. Editing Parameters.

In [15]:
print(spb_sfx.params['d1']) ### the parameters file is loaded as a dictionary - this step gives us all the available parameters to change.
spb_sfx.params['d1']['distance'] = 247 ### note, changing this drift distance does not affect future drift distances (this is a to-do).
spb_sfx.rebuild()


{'name': 'd1', 'description': 'Drift from Source to HOM1', 'distance': 247, 'pp': [1, 2.5, 1, 2.5, 'quadratic']}


## Extracting the Beamline object
Finally, we want to extract the wpg beamline object for use in simulation.

In [17]:
bl = spb_sfx.bl

author: twguest